In [8]:
from dataset import tweetProccess
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk, re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.sentiment.vader import SentimentIntensityAnalyzer as si


'''gets the data'''
dataHandler = tweetProccess.FileTreat()
content = dataHandler.getTweets()

no_features = 2000


#Use to download nltk content
#nltk.download()

'''defining a function to remove numbers'''
def tokenize(text):
    tokenized = []
    for word in nltk.word_tokenize(text):
        if re.search('[a-zA-Z]', word):
            tokenized.append(word.lower())
            
    return tokenized

'''defining a function to tokenize and get only vebs and adjectivies'''
def tokenizeStructures(text):
    #gets the type of each word in the text
    tags = pos_tag(word_tokenize(text))
    
    interesting = []
    for tag in tags:
        #removes tweet links
        if re.search('t.co+', tag[0]):
            continue
        
        if (tag[1] == 'VERB' or tag[1] == 'JJ') and len(tag[0]) > 2:
            interesting.append(tag[0])
    
    return interesting

def getLda(dataFrame, no_clusters):
    ldas = []
    no_topics = 3
    
    #for each cluster
    print(no_clusters)
    for cluster in range(no_clusters):
        print('cluster: ' + str(cluster))
        
        lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
        tf_vectorizer = CountVectorizer(tokenizer=tokenizeStructures, max_features=no_features, stop_words='english')
        
        #get text of tweets from this cluster
        texts = dataFrame['text'][cluster].values
        
        #creates a tf matrix for this cluster
        tf = tf_vectorizer.fit_transform(texts)
        
        #get feature names for this matrix
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        #extract latent topics
        ldas.append((cluster, lda.fit(tf), tf_feature_names))
        
    return ldas
        
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
        
def get_neg_matrix(texts):
    analyser = si()
    vec = []
    #print(texts)
    for sentense in texts:
        obj = analyser.polarity_scores(sentense)
        #vec.append([analyser.polarity_scores(sentense)['neg']])
        if(obj is not None):
            vec.append([obj['neg'], obj['pos']])
    return vec



In [10]:
from sklearn.cluster import KMeans

#generate the tf matrix
#tf_vectorizer = CountVectorizer(tokenizer=tokenizeStructures, max_features=no_features, stop_words='english')
#tf = tf_vectorizer.fit_transform(content)

#gets the negative matrix

#generates the clusters
n_cluters = 3
km = KMeans(n_clusters = n_cluters)
#%time km.fit(tf)
%time km.fit(get_neg_matrix(content))


clusters = km.labels_.tolist()

CPU times: user 1min, sys: 76 ms, total: 1min
Wall time: 59.2 s


In [11]:
import pandas as pd

table = {'text': content, 'cluster': clusters, 'id': [idx for idx in range(len(content))]}

dataframe = pd.DataFrame(table, index=[clusters], columns=['text', 'cluster', 'id'])

#sizes of each cluster
print(dataframe['cluster'].value_counts())

#gets the latent topic of each cluster
result = getLda(dataframe, n_cluters)

0    100624
1     47407
2     30003
Name: cluster, dtype: int64
3
cluster: 0
cluster: 1
cluster: 2


In [12]:
no_top_words = 10

for cluster, lda, feature_names in result:
    #print(len(lda.components_[0]))
    if(len(feature_names) > 10):
        
        print("cluster: " + str(cluster))
        print_top_words(lda, feature_names, 10)
        print('================================================')

cluster: 0
Topic #0: little real mental big right white sure bad able ptsd
Topic #1: ive dont hard old fat long happy different black high
Topic #2: new good single social live mindful young free fucking second
cluster: 1
Topic #0: happy good beautiful true ive dont awesome ready real cute
Topic #1: free amazing important little wish alive hard bad positive wonderful
Topic #2: great new strong perfect nice sure favorite proud easy dear
cluster: 2
Topic #0: bad dead mental sick tired new fucking real fake old
Topic #1: hard dont fuck little sorry suicidal pathetic social long horrible
Topic #2: sad good wrong ive stupid happy ill fat shit broken


In [13]:
#print(enumerate(result[3][1].components_))

In [14]:
dataframe.tail()

,text,cluster,id
0,I still stand by this. If youre having a shit ...,0,178029
0,RT @machinegunkelly: the news cant be true. iv...,0,178030
1,RT @machinegunkelly: love you Chester. love yo...,1,178031
1,Linkin Park was a huge part of my childhood. D...,1,178032
0,RT @That70sFans: can we just talk about how ro...,0,178033


In [15]:
dataframe['cluster'].value_counts()
dataframe.tail()


,text,cluster,id
0,I still stand by this. If youre having a shit ...,0,178029
0,RT @machinegunkelly: the news cant be true. iv...,0,178030
1,RT @machinegunkelly: love you Chester. love yo...,1,178031
1,Linkin Park was a huge part of my childhood. D...,1,178032
0,RT @That70sFans: can we just talk about how ro...,0,178033


In [16]:
#from sklearn.metrics.pairwise import cosine_similarity
#dist = 1 - cosine_similarity(tf)

#print(dist)